# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Thu 2023-01-12 21:00:44.230947 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE
Thu 2023-01-12 21:00:44.231443 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
Thu 2023-01-12 21:00:44.231520 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Thu 2023-01-12 21:00:44.233514 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Thu 2023-01-12 21:00:44.256412 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fe4dd532b70>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7fe4dd8a6fc0>, <FilterInterface at 0x7fe4dd8a6d40>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient5526", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient5526", client_version: "0.0.666" } }

In [12]:
await client_fiface.command_response(filter_cmd)

Thu 2023-01-12 21:01:07.410600 [TRACE] - [F<-AC] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient5526", client_version: "0.0.666" } }
LTI(client)192 skipped 191: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
Thu 2023-01-12 21:01:07.413413 [TRACE] - [T<-FC] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
LTI(client)193: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:07.416512 [TRACE] - [F<-TR] Command Accepted
LTI(client)194: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5526", client_version: "0.0.666" } }] }
Thu 2023-01-12 21:01:07.417331 [TRACE] - [T<-FC] Proc

Accepted

LTI(client)198: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:07.496452 [TRACE] - [F<-TR] Command Accepted
LTI(client)199: got transport notifications: []
LTI(client)224 skipped 24: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-12 21:01:10.017351 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-12 21:01:10.017505 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: None, action: KeepAlive { la

In [13]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: OK }

LTI(client)258 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 4, response_ack: Some(3), cookie: None, action: KeepAlive { latest_response_ack: 3 } }] }
Thu 2023-01-12 21:01:12.646795 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 4, response_ack: Some(3), cookie: None, action: KeepAlive { latest_response_ack: 3 } }] }
Thu 2023-01-12 21:01:12.646903 [TRACE] - [T<-UDP] Request { sequence: 4, response_ack: Some(3), cookie: None, action: KeepAlive { latest_response_ack: 3 } }
LTI(client)259: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:12.647495 [TRACE] - [F<-TR] Command Accepted
Thu 2023-01-12 21:01:12

In [14]:
cookie = new_response_code.response_code.cookie
cookie

LTI(client)265: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 5, request_ack: Some(5), code: OK } }]
LTI(client)266: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 6 } }
LTI(client)267: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:12.850183 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 5, request_ack: Some(5), code: OK }
Thu 2023-01-12 21:01:12.851383 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 6 } }
Thu 2023-01-12 21:01:12.851823 [TRACE] - [F<-TR] Command Accepted
LTI(client)268: got transport notifications: []
LTI(client)292 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { 

LTI(client)333: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 9, request_ack: Some(9), code: OK } }]
Thu 2023-01-12 21:01:18.116239 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 9, request_ack: Some(9), code: OK }
LTI(client)334: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 10 } }
Thu 2023-01-12 21:01:18.116957 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 10 } }
LTI(client)335: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:18.117305 [TRACE] - [F<-TR] Command Accepted
LTI(client)336: got transport notifications: []
LTI(client)360 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId 

LTI(client)394 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 11 }, retry_interval: 250ms }], packets: [Request { sequence: 12, response_ack: Some(11), cookie: None, action: KeepAlive { latest_response_ack: 11 } }] }
Thu 2023-01-12 21:01:23.175400 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 11 }, retry_interval: 250ms }], packets: [Request { sequence: 12, response_ack: Some(11), cookie: None, action: KeepAlive { latest_response_ack: 11 } }] }
LTI(client)395: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:23.175497 [TRACE] - [T<-UDP] Request { sequence: 12, response_ack: Some(11), cookie: None, action: KeepAlive { latest_response_ack: 11 } }
Thu 2023-01-12 21:01:23.175916 [TRACE] - [F<-TR] Command Accepted
Thu 2023-01-

LTI(client)435: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 15, request_ack: Some(15), code: OK } }]
Thu 2023-01-12 21:01:26.016371 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 15, request_ack: Some(15), code: OK }
LTI(client)436: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 13 } }
Thu 2023-01-12 21:01:26.017090 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 13 } }
LTI(client)437: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:26.017382 [TRACE] - [F<-TR] Command Accepted
LTI(client)438: got transport notifications: []
LTI(client)462 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqu

LTI(client)503: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 19, request_ack: Some(19), code: OK } }]
Thu 2023-01-12 21:01:31.285928 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 19, request_ack: Some(19), code: OK }
LTI(client)504: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 9 } }
Thu 2023-01-12 21:01:31.286461 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 9 } }
LTI(client)505: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:31.286988 [TRACE] - [F<-TR] Command Accepted
LTI(client)506: got transport notifications: []
LTI(client)530 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueI

...

LTI(client)605: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 25, request_ack: Some(25), code: OK } }]
Thu 2023-01-12 21:01:39.180599 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 25, request_ack: Some(25), code: OK }
LTI(client)606: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 18 } }
Thu 2023-01-12 21:01:39.182735 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 18 } }
LTI(client)607: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:39.183634 [TRACE] - [F<-TR] Command Accepted
LTI(client)608: got transport notifications: []
LTI(client)632 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqu

LTI(client)666 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 15 }, retry_interval: 250ms }], packets: [Request { sequence: 28, response_ack: Some(27), cookie: None, action: KeepAlive { latest_response_ack: 27 } }] }
Thu 2023-01-12 21:01:44.236120 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 15 }, retry_interval: 250ms }], packets: [Request { sequence: 28, response_ack: Some(27), cookie: None, action: KeepAlive { latest_response_ack: 27 } }] }
Thu 2023-01-12 21:01:44.236427 [TRACE] - [T<-UDP] Request { sequence: 28, response_ack: Some(27), cookie: None, action: KeepAlive { latest_response_ack: 27 } }
LTI(client)667: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:44.237391 [TRACE] - [F<-TR] Command Accepted
Thu 2023-01-

LTI(client)707: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 31, request_ack: Some(31), code: OK } }]
Thu 2023-01-12 21:01:47.070991 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 31, request_ack: Some(31), code: OK }
LTI(client)708: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 20 } }
Thu 2023-01-12 21:01:47.071520 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 20 } }
LTI(client)709: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:47.071663 [TRACE] - [F<-TR] Command Accepted
LTI(client)710: got transport notifications: []
LTI(client)734 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqu

LTI(client)768 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 34, response_ack: Some(33), cookie: None, action: KeepAlive { latest_response_ack: 33 } }] }
Thu 2023-01-12 21:01:52.128938 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 34, response_ack: Some(33), cookie: None, action: KeepAlive { latest_response_ack: 33 } }] }
Thu 2023-01-12 21:01:52.129313 [TRACE] - [T<-UDP] Request { sequence: 34, response_ack: Some(33), cookie: None, action: KeepAlive { latest_response_ack: 33 } }
LTI(client)769: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:52.129992 [TRACE] - [F<-TR] Command Accepted
Thu 2023-01-12

LTI(client)809: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Response { sequence: 37, request_ack: Some(37), code: OK } }]
Thu 2023-01-12 21:01:54.965296 [TRACE] - [F<-TN] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 37, request_ack: Some(37), code: OK }
LTI(client)810: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 26 } }
Thu 2023-01-12 21:01:54.965827 [TRACE] - [T<-FC] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 2, offset: 26 } }
LTI(client)811: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:54.966078 [TRACE] - [F<-TR] Command Accepted
LTI(client)812: got transport notifications: []


In [15]:
cookie

LTI(client)836 skipped 23: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 24 }, retry_interval: 250ms }], packets: [Request { sequence: 38, response_ack: Some(37), cookie: None, action: KeepAlive { latest_response_ack: 37 } }] }
Thu 2023-01-12 21:01:57.394689 [TRACE] - [T<-FC] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 24 }, retry_interval: 250ms }], packets: [Request { sequence: 38, response_ack: Some(37), cookie: None, action: KeepAlive { latest_response_ack: 37 } }] }
Thu 2023-01-12 21:01:57.394910 [TRACE] - [T<-UDP] Request { sequence: 38, response_ack: Some(37), cookie: None, action: KeepAlive { latest_response_ack: 37 } }
LTI(client)837: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-12 21:01:57.395991 [TRACE] - [F<-TR] Command Accepted
Thu 2023-01-

In [ ]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
server_notifications[0]

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()